In [1]:
import string
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
all_chars       = string.printable
n_chars         = len(all_chars)
file            = open('./cnus.txt').read()
file_len        = len(file)

print('Length of file: {}'.format(file_len))
print('All possible characters: {}'.format(all_chars))
print('Number of all possible characters: {}'.format(n_chars))

Length of file: 3381928
All possible characters: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

Number of all possible characters: 100


In [3]:
# Remove all non-printable characters
file = ''.join(list(filter((lambda x: x in all_chars), file)))

In [4]:
file_len = len(file)
print('Length of file: {}'.format(file_len))

Length of file: 3381831


In [5]:
# Get a random sequence of the Sherlock dataset.
def get_random_seq():
    seq_len     = 128  # The length of an input sequence.
    start_index = random.randint(0, file_len - seq_len)
    end_index   = start_index + seq_len + 1
    return file[start_index:end_index]

# Convert the sequence to one-hot tensor.
def seq_to_onehot(seq):
    tensor = torch.zeros(len(seq), 1, n_chars) 
    # Shape of the tensor:
    #     (sequence length, batch size, classes)
    # Here we use batch size = 1 and classes = number of unique characters.
    for t, char in enumerate(seq):
        index = all_chars.index(char)
        tensor[t][0][index] = 1
    return tensor

# Convert the sequence to index tensor.
def seq_to_index(seq):
    tensor = torch.zeros(len(seq), 1)
    # Shape of the tensor: 
    #     (sequence length, batch size).
    # Here we use batch size = 1.
    for t, char in enumerate(seq):
        tensor[t] = all_chars.index(char)
    return tensor

# Sample a mini-batch including input tensor and target tensor.
def get_input_and_target():
    seq    = get_random_seq()
    input  = seq_to_onehot(seq[:-1])      # Input is represented in one-hot.
    target = seq_to_index(seq[1:]).long() # Target is represented in index.
    return input, target

In [6]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  
# If 'cuda:0' is printed, it means GPU is available.

cuda:0


In [7]:
class Net(nn.Module):
    def __init__(self, activation='RNN'):
        # Initialization.
        super(Net, self).__init__()
        self.input_size  = n_chars   # Input size: Number of unique chars.
        self.hidden_size = 100       # Hidden size: 100.
        self.output_size = n_chars   # Output size: Number of unique chars.
        
        self.linear = nn.Linear(self.hidden_size, self.output_size)
        if activation == 'RNN':
            self.activation = nn.RNNCell(self.input_size, self.hidden_size)
        elif activation == 'LSTM':
            self.activation = nn.LSTMCell(self.input_size, self.hidden_size)
        
        self.act = activation
            
    def forward(self, input, hidden):
        """ Forward function.
              input:  One-hot input. It refers to the x_t in homework write-up.
              hidden: Previous hidden state. It refers to the h_{t-1}.
            Returns (output, hidden) where output refers to y_t and 
                     hidden refers to h_t.
        """
        if self.act == 'RNN':
            hidden = self.activation(input, hidden)
            output = self.linear(hidden)
            return output, hidden
        else:
            hidden, cell = self.activation(input, hidden)
            output = self.linear(hidden)
            return output, hidden, cell

    def init_hidden(self):
        # Initial hidden state.
        # 1 means batch size = 1.
        return torch.zeros(1, self.hidden_size).to(device) 
    
    def init_cell(self):
        # Initial cell state.
        # 1 means batch size = 1.
        return torch.zeros(1, self.hidden_size).to(device) 

In [8]:
# Training step function.
def train_step(net, opt, input, target):
    """ Training step.
        net:    The network instance.
        opt:    The optimizer instance.
        input:  Input tensor.  Shape: [seq_len, 1, n_chars].
        target: Target tensor. Shape: [seq_len, 1].
    """
    seq_len = input.shape[0]    # Get the sequence length of current input.
    hidden = net.init_hidden()  # Initial hidden state.
    cell = net.init_cell()      # Initial cell state
    net.zero_grad()             # Clear the gradient.
    loss = 0                    # Initial loss.
    
    if net.act == 'RNN':
        for t in range(seq_len):    # For each one in the input sequence.
            output, hidden = net(input[t], hidden)
            loss += loss_func(output, target[t])
    else:
        for t in range(seq_len):    # For each one in the input sequence.
            output, hidden, cell = net(input[t], (hidden, cell))
            loss += loss_func(output, target[t])

    loss.backward()             # Backward. 
    opt.step()                  # Update the weights.

    return loss / seq_len       # Return the average loss w.r.t sequence length.

In [9]:
# Evaluation step function.
def eval_step(net, init_seq='W', predicted_len=100):
    # Initialize the hidden state, input and the predicted sequence.
    hidden        = net.init_hidden()
    cell          = net.init_cell()
    init_input    = seq_to_onehot(init_seq).to(device)
    predicted_seq = init_seq

    # If input net is RNN
    if net.act == 'RNN':
        # Use initial string to "build up" hidden state.
        for t in range(len(init_seq) - 1):
            output, hidden = net(init_input[t], hidden)
        # Set current input as the last character of the initial string.
        input = init_input[-1]
        
        # Predict more characters after the initial string.
        for t in range(predicted_len):
            # Get the current output and hidden state.
            output, hidden = net(input, hidden)
        
            # Sample from the output as a multinomial distribution.
            predicted_index = torch.multinomial(output.view(-1).exp(), 1)[0]
        
            # Add predicted character to the sequence and use it as next input.
            predicted_char  = all_chars[predicted_index]
            predicted_seq  += predicted_char
        
            # Use the predicted character to generate the input of next round.
            input = seq_to_onehot(predicted_char)[0].to(device)
    
    # If input net is LSTM       
    else:
        # Use initial string to "build up" hidden state.
        for t in range(len(init_seq) - 1):
            output, hidden, cell = net(init_input[t], (hidden, cell))
        # Set current input as the last character of the initial string.
        input = init_input[-1]
        
        # Predict more characters after the initial string.
        for t in range(predicted_len):
            # Get the current output and hidden state.
            output, hidden, cell = net(input, (hidden, cell))
        
            # Sample from the output as a multinomial distribution.
            predicted_index = torch.multinomial(output.view(-1).exp(), 1)[0]
        
            # Add predicted character to the sequence and use it as next input.
            predicted_char  = all_chars[predicted_index]
            predicted_seq  += predicted_char
        
            # Use the predicted character to generate the input of next round.
            input = seq_to_onehot(predicted_char)[0].to(device)

    return predicted_seq

### Experiment

In [10]:
# Perform n_trials each for standard RNN and LSTM.

n_trials    = 3      # Number of trials per model
iters       = 20000  # Number of training iterations per trial.
print_iters = 100    # Number of iterations for each log printing.

In [11]:
# The loss variables.
all_losses_rnn = np.zeros(int(iters / print_iters))

# Training procedure.
for i in range(n_trials):
    net_rnn = Net()     # Create a new network instance.
    net_rnn.to(device)
    opt_rnn       = torch.optim.Adam(net_rnn.parameters(), lr=0.005)
    loss_func = nn.CrossEntropyLoss()
    losses = np.array([])
    loss_sum = 0
    for j in range(iters):
        input, target = get_input_and_target()            # Fetch input and target.
        input, target = input.to(device), target.to(device) # Move to GPU memory.
        loss      = train_step(net_rnn, opt_rnn, input, target)   # Calculate the loss.
        loss_sum += loss.item()                                  # Accumulate the loss.

        # Print the log.
        if j % print_iters == print_iters - 1:
            print('rnn_trial:{}, iter:{}/{} loss:{}'.format(i, j, iters, loss_sum / print_iters))
            print('generated sequence: {}\n'.format(eval_step(net_rnn)))
              
            # Track the loss.
            losses = np.append(losses, loss_sum / print_iters)
            loss_sum = 0
    all_losses_rnn += losses
    
mean_losses_rnn = all_losses_rnn / n_trials

rnn_trial:0, iter:99/20000 loss:3.065303633213043
generated sequence: W,tpo f enhoyitIt e hets ard grinnnepnoonsnkuts isfnaafdotar  s e hndprd., as odmelI"aeFepse sas tt.M

rnn_trial:0, iter:199/20000 loss:2.793942792415619
generated sequence: Wlede thiretgabmeot  ts
 s rYdvmbits
w  rpf"? aneercld"yuglou
 thelx
e"sy, mi raa, sort ontitsluwland

rnn_trial:0, iter:299/20000 loss:2.540985517501831
generated sequence: Wln ce or is ang ant   
  oad, ?  oly mer bsest
 atounlgt   pin, somoue mere. p. and sn yintat i!o
  

rnn_trial:0, iter:399/20000 loss:2.3971801710128786
generated sequence: Wsh-duy uncI thedterimis wotle dus wek
 f
    we "thisytwad tued I
    " toupunnse gag"

   uvery ets

rnn_trial:0, iter:499/20000 loss:2.355198042392731
generated sequence: Wcor. teroflkowas ir ond wanoryes the
     torit "     Thermanitdt oortive houllor. Wois, mimerere y 

rnn_trial:0, iter:599/20000 loss:2.24158495426178
generated sequence: We mrir."


   "Butsregs than'y. Thoul.


    arom
   ther n

rnn_trial:0, iter:4799/20000 loss:1.8205506205558777
generated sequence: Wethe home oug My. Hey vellom put have had
     nots parder to hellking howe, Gurge fat of tustamate 

rnn_trial:0, iter:4899/20000 loss:1.778361839056015
generated sequence: Wh,
     which sow" he dloight. He arriegite
     chelling simered is a dast atanist. He wall. It not

rnn_trial:0, iter:4999/20000 loss:1.7793945932388306
generated sequence: Wh hewrioge pawne."

     "It to this, about I endestiopatty poingarcone criet thop
     spethers and

rnn_trial:0, iter:5099/20000 loss:1.8017090356349945
generated sequence: Wanto
     ftulion ophaltlich. What wa5t have to I our that the me?"

     "Then a lees, oat, When wh

rnn_trial:0, iter:5199/20000 loss:1.833299686908722
generated sequence: Wely made yes, Sin beenget aldingn, his camenced nots desterplicose mer thinking reange, seorse-sife 

rnn_trial:0, iter:5299/20000 loss:1.8213476097583772
generated sequence: Wat frouch to ald to there my cord. I
     hat, h

rnn_trial:0, iter:9499/20000 loss:2.0383488047122955
generated sequence: What to ere seppave a vain thrat.    But a brot with see
   glale. to saukes in!" said some coulvert 

rnn_trial:0, iter:9599/20000 loss:1.8460715079307557
generated sequence: When on ever jessens wick over-on, not gansing
     Lary Uhing. Thatcen
    "Planes. "Thenk amcaicief

rnn_trial:0, iter:9699/20000 loss:1.8513566184043884
generated sequence: WER" cousided sampllestlespishation, and jnably oneygate holsed
     topesed windon so voPlerevisceni

rnn_trial:0, iter:9799/20000 loss:1.7759202551841735
generated sequence: Whose norlucs. But werh!
     Sole good the with more thougmed of
     s," say
     exprocennd undoor

rnn_trial:0, iter:9899/20000 loss:1.7786919903755187
generated sequence: Whericarcisioneld
     his cout to ont, whece be vare apped ly,"

     "He rays, o condricts
     me 

rnn_trial:0, iter:9999/20000 loss:1.7402017199993134
generated sequence: Wrees tert. It wat coung of the warlls."

     

rnn_trial:0, iter:14199/20000 loss:1.7106503796577455
generated sequence: We." he dowall furning, we thow of the
     on Sir Holmes by a murper, and should tereven with a leai

rnn_trial:0, iter:14299/20000 loss:1.697185845375061
generated sequence: Whtyoot from nrum norestry. I has coefetporturd wording tearco dor's in tel giaiction hauttiling to d

rnn_trial:0, iter:14399/20000 loss:1.7206688225269318
generated sequence: Whime which was a
     natenclow it
     recoscere-lyas. ID is flet himshal in
     cas, with they ho

rnn_trial:0, iter:14499/20000 loss:1.740632997751236
generated sequence: We."

     "It supprounot wife to Cut,
     had these nast as if ser; anbened
     grearn figlul mest

rnn_trial:0, iter:14599/20000 loss:1.6981585502624512
generated sequence: We had bricle rill dear wave nresen
     be there pluble but over he. I otle so, brle.

     "Hosel, 

rnn_trial:0, iter:14699/20000 loss:1.7349661087989807
generated sequence: We could had to
     of year out liak. At I

rnn_trial:0, iter:18899/20000 loss:1.7435402429103852
generated sequence: Walked brimuse endy live was eftery a the hid deakionsone he car watconedy neccapse modechilh you are

rnn_trial:0, iter:18999/20000 loss:1.739739887714386
generated sequence: Whtrror,, I comsangient, in the mack which, but the waad have me, and ingenpang of the nototon't real

rnn_trial:0, iter:19099/20000 loss:1.8499766194820404
generated sequence: Whole with he really the Parks, slooro- if the Pemblow-drised it me is brew--and is was this can
    

rnn_trial:0, iter:19199/20000 loss:1.7750451910495757
generated sequence: Whlaconark and of his sair him my bay lle, of the of he out othibe; I offoom coulow you houbhions of 

rnn_trial:0, iter:19299/20000 loss:1.723585206270218
generated sequence: Welld me was consideie, pade thre knoot the acced clurain that yat eneace, sevent.   rormell long his

rnn_trial:0, iter:19399/20000 loss:1.7236552000045777
generated sequence: We, we seat ass
     to my know. And the we

...

rnn_trial:1, iter:17699/20000 loss:1.6259630310535431
generated sequence: Wat white not Ledmity, and my once
     one
     theat, and you wancherg minting."

     "It's colfer

rnn_trial:1, iter:17799/20000 loss:1.6757045340538026
generated sequence: We amself.-He said the poct a h9s possien came arrard; and chough and place, sower, work d of the was

rnn_trial:1, iter:17899/20000 loss:1.6439367997646332
generated sequence: Whinkeng her that's fintinging had This swid, which housers at onwe wouled reluston as introom
     h

rnn_trial:1, iter:17999/20000 loss:1.7081813955307006
generated sequence: We thith of
     by the wondon arrimant
     and the hald the stormor. "I have should himrer, this fi

rnn_trial:1, iter:18099/20000 loss:1.6709110736846924
generated sequence: What I have the one inconle pression.'

     "The policy? Hell an and ficton him doing mour. Axticy o

rnn_trial:1, iter:18199/20000 loss:1.6950150179862975
generated sequence: Wats, when the Amohed that the exampoor w

rnn_trial:2, iter:2399/20000 loss:1.8737804675102234
generated sequence: Wel--sunftid.  he shopely.

     It wouth the tantollagery at comftornoseroun appexsopen of ly any ca

rnn_trial:2, iter:2499/20000 loss:1.857670978307724
generated sequence: Wifkem his resveprass busend, word a cources limptanle Sfong M s extreed sokparred and worl wele ferm

rnn_trial:2, iter:2599/20000 loss:1.8234456181526184
generated sequence: Well erisbe is made ets tir
     that itt exinnow Wisht it, Ms 'le, you sers.

     "Sexund surg we p

rnn_trial:2, iter:2699/20000 loss:1.8287354612350464
generated sequence: We ans wau had agh to deade the had ssent overs with if a stoply his nefurge this upon you narver by 

rnn_trial:2, iter:2799/20000 loss:1.8049019527435304
generated sequence: We nevarns flatt you hore af broty your eyon severive frry any insce to thing; and her, and shat a so

rnn_trial:2, iter:2899/20000 loss:1.8189448165893554
generated sequence: We lengare pore of you an
     letled that
     

rnn_trial:2, iter:7099/20000 loss:1.7405814278125762
generated sequence: When om!"

     Lernes
     to cound upintion mesion
     ampentle. loit it it, find. Gut nut hand. I

rnn_trial:2, iter:7199/20000 loss:1.7309138894081115
generated sequence: We seenly
 g tomain to that is deeving

     The litter Pe and hith mive that must of he tome more th

rnn_trial:2, iter:7299/20000 loss:1.7412806618213654
generated sequence: We excuurs of the he had I lay the ald that oicle onqyore, all rath the under that or hare of thene a

rnn_trial:2, iter:7399/20000 loss:1.7102324509620666
generated sequence: Well, and critten nvil the a
     whinged
     spol-whitaly secounc
     wat bsen remppping his

    

rnn_trial:2, iter:7499/20000 loss:1.7043892455101013
generated sequence: We to betrom belant of know, as Lomto
     fivered inten of the man I wipes this a cameted vamaled ex

rnn_trial:2, iter:7599/20000 loss:1.7341500270366668
generated sequence: Well Run weich the rid oy had heark to deapt la

rnn_trial:2, iter:11799/20000 loss:1.7110033118724823
generated sequence: We dlan for fount-at. We atlo shoudd in the fair so ispolren to been would notto to an intembence. Th

rnn_trial:2, iter:11899/20000 loss:1.698392949104309
generated sequence: West somuse aut your seceo at
     a flares in it, not had is had some anr, she take at somenter stap

rnn_trial:2, iter:11999/20000 loss:1.7051066374778747
generated sequence: Wang actobed lasting this of my hat was but of anken pleited, cain liter,
     but eation, as las lay

rnn_trial:2, iter:12099/20000 loss:1.6827387499809265
generated sequence: We shoke me stortions Helosh by which you is scrick in, I
     all, whilooth it it to dent
     assed

rnn_trial:2, iter:12199/20000 loss:1.6786078000068665
generated sequence: We upon. I fumation to con'itall the lask eat shipon. A san I wonl. I syous to the sperite- Lon, nut 

rnn_trial:2, iter:12299/20000 loss:1.7103424739837647
generated sequence: Well the crofe of his Ind the didected exc

rnn_trial:2, iter:16499/20000 loss:1.6803698885440825
generated sequence: We wank should I
     we estick Lors, man, was not of
     nhes under,
     dotedpaclly out. There it

rnn_trial:2, iter:16599/20000 loss:1.672696145772934
generated sequence: We waterettennay I could, of priseler, and been accorranceap, and the
     overy eitumard, buttrecore

rnn_trial:2, iter:16699/20000 loss:1.690032538175583
generated sequence: When even? When S:, by aine len sevelf the ect the Grnef's
     shey hers a quefers interacounce up t

rnn_trial:2, iter:16799/20000 loss:1.6960233855247497
generated sequence: We excurlate blook og heloskance round."

     "if overmiblice."

     "Why out ede puggh my day Cony

rnn_trial:2, iter:16899/20000 loss:1.674116085767746
generated sequence: We would
     all that I dodend he was frenother, three ascance. Why witkect who
     you; I stite we

rnn_trial:2, iter:16999/20000 loss:1.6893559885025025
generated sequence: We fate mysmented and of tholk
     a could'

In [12]:
all_losses_lstm = np.zeros(int(iters / print_iters))

for i in range(n_trials):
    net_lstm = Net('LSTM')    # Create a new network instance.
    net_lstm.to(device)
    opt_lstm      = torch.optim.Adam(net_lstm.parameters(), lr=0.005)
    loss_func = nn.CrossEntropyLoss()
    losses = np.array([])
    loss_sum = 0
    for j in range(iters):
        input, target = get_input_and_target()            # Fetch input and target.
        input, target = input.to(device), target.to(device) # Move to GPU memory.
        loss      = train_step(net_lstm, opt_lstm, input, target)   # Calculate the loss.
        loss_sum += loss.item()                                  # Accumulate the loss.

        # Print the log.
        if j % print_iters == print_iters - 1:
            print('lstm_trial:{}, iter:{}/{} loss:{}'.format(i, j, iters, loss_sum / print_iters))
            print('generated sequence: {}\n'.format(eval_step(net_lstm)))
              
            # Track the loss.
            losses = np.append(losses, loss_sum / print_iters)
            loss_sum = 0
    all_losses_lstm += losses

mean_losses_lstm = all_losses_lstm / n_trials

lstm_trial:0, iter:99/20000 loss:3.055918459892273
generated sequence: W.M gs sth ea x le y f uet hsihnaa
un  h!tnmtau"eh we hereH afalathh dlu c,twvp.eotl  m   tothtrts lr

lstm_trial:0, iter:199/20000 loss:2.671723268032074
generated sequence: Wab tthoumihe  ohe ne ae. bafcbrinclned wminnec, he w uh  uv   .he  th   mhar Are s d ocach
 hor andh

lstm_trial:0, iter:299/20000 loss:2.3949028491973876
generated sequence: Wdeke,d ir uanded
       f

      "
     ove wtatd. and Hee, shel,

     eglincitb, ve hon I t" tanc.

lstm_trial:0, iter:399/20000 loss:2.2435578322410583
generated sequence: W doclt an thacw

      y wam thu
g    "of

    I frand ang simast thif dous toul ou ttoAd berin.  at

lstm_trial:0, iter:499/20000 loss:2.146622824668884
generated sequence: W!ewt

     I ha
     y seo, lnomen whto ay d thure.

     Watat at on y asg whos I thiw deamk mhipe


lstm_trial:0, iter:599/20000 loss:2.0564072799682616
generated sequence: W
     sisreabr nouz of mit jitunny
     forsangnin

lstm_trial:0, iter:4799/20000 loss:1.5958861005306244
generated sequence: Wo Long
     some klows not centialy ir the said to detut,
     cat?"

     "Had by I thought could n

lstm_trial:0, iter:4899/20000 loss:1.6065684723854066
generated sequence: Waus int me. Firsiles, round
     Janunts to orhtigule sefiem from whichad lan theys coverating, and 

lstm_trial:0, iter:4999/20000 loss:1.600703434944153
generated sequence: Whough he was no to mer the fointy by kater."

     "What was a piskery nothoper; raid littgenantictl

lstm_trial:0, iter:5099/20000 loss:1.562001601457596
generated sequence: Way not lengising that we prise will was engeple of
     with being sour nark."

     "He.

     The 

lstm_trial:0, iter:5199/20000 loss:1.5734630584716798
generated sequence: What yet
     Swerlete grows. There. Though the roofs in the more he reget. I countered vade man over

lstm_trial:0, iter:5299/20000 loss:1.5899572741985322
generated sequence: Went rexach intere?"

     "Was came and
  

lstm_trial:0, iter:9499/20000 loss:1.476186454296112
generated sequence: What it was he wark that we morney. His dread of his sthrell me, but you aity all. He wasted by the d

lstm_trial:0, iter:9599/20000 loss:1.48853741645813
generated sequence: Wefe to the roumid.
     You sake on my sunslace all wherph anythings in twends a bell
     which not

lstm_trial:0, iter:9699/20000 loss:1.4811912739276887
generated sequence: When McCarta. For such eyecredved and
     winds to myst when I
     heard ir
     I side of mis, for

lstm_trial:0, iter:9799/20000 loss:1.5038634729385376
generated sequence: Wife where I foldingservalle earjediun warker after the trist.

     "Yes. But herebood has might rea

lstm_trial:0, iter:9899/20000 loss:1.4539796221256256
generated sequence: With a pressered my room of seart and going dishound you have an informed to in't the trupped up his 

lstm_trial:0, iter:9999/20000 loss:1.4912102007865906
generated sequence: We appertom on the screwing soved simind?"



lstm_trial:0, iter:14199/20000 loss:1.4436979162693024
generated sequence: We spule. As his shorous
     anmaking our droaks of us, good. "You
     crime, too knook that he had

lstm_trial:0, iter:14299/20000 loss:1.488018970489502
generated sequence: Wife?'

     As Sir Courso-feely hand apseminey are the visitly compaaled, It
     read mugray eviden

lstm_trial:0, iter:14399/20000 loss:1.4519011640548707
generated sequence: Well,
     Tretely. But if he had a corced marme and before friding-side, and have my
     rue to kee

lstm_trial:0, iter:14499/20000 loss:1.4391104590892791
generated sequence: Well, but for we saw shinded in withans his conninted. Out agabp of our findon that you have the wait

lstm_trial:0, iter:14599/20000 loss:1.4539951848983765
generated sequence: Woflowed
     difficult down and out of the momine in a
     so. But I recusal and wommer was."

    

lstm_trial:0, iter:14699/20000 loss:1.4325560569763183
generated sequence: Wrifouldicion fashion, weech
     wi

lstm_trial:0, iter:18899/20000 loss:1.395303294658661
generated sequence: Womered all frome would be clearlived me, Mr.
     Holmes attent, when adversibular one amint acfulti

lstm_trial:0, iter:18999/20000 loss:1.4048327124118805
generated sequence: We stepp ourselves of these. Only did so did you's he
     a quairon among, Mr. Herrints which ring o

lstm_trial:0, iter:19099/20000 loss:1.4278268253803252
generated sequence: With the wore, someluned to my left that she lady e, the thing to you mountick, excutach every serieb

lstm_trial:0, iter:19199/20000 loss:1.4055282139778138
generated sequence: Wasted through, but this. Are there were the rigg chaition to a deepent ask of the tragive and jusige

lstm_trial:0, iter:19299/20000 loss:1.4108854746818542
generated sequence: Wall two show,
     young stopped it a very position again whom you sixte's from the passed, who was 

lstm_trial:0, iter:19399/20000 loss:1.409529214501381
generated sequence: Wathopp. I had not was
     driving a

lstm_trial:1, iter:3599/20000 loss:1.6717776620388032
generated sequence: When io a
     which congose withous un."


                          mustly sunder.
     Lecood Lene

lstm_trial:1, iter:3699/20000 loss:1.680481083393097
generated sequence: Wull dost by a was
     bitting carpsades Holmes. Houd has he glard, sil Adalt thinl difoth. Pady str

lstm_trial:1, iter:3799/20000 loss:1.6658576571941375
generated sequence: When your fruenes upon a
    "Utellife. Raay clooked
     sook!"

     "You meand then the
     torus

lstm_trial:1, iter:3899/20000 loss:1.6544576346874238
generated sequence: We?"

     "We assered,
     us the peerten, an't Laty a wastry bacter my now conter'ing a whone of t

lstm_trial:1, iter:3999/20000 loss:1.6383541250228881
generated sequence: When been around live start which had neve to been aboul most in his plack me. I was moend on thrindd

lstm_trial:1, iter:4099/20000 loss:1.6444614231586456
generated sequence: Whoteg we
     rive's were fof mosting of 

lstm_trial:1, iter:8299/20000 loss:1.524587391614914
generated sequence: What I puth a am mester."

     "Macaous ."Yes.

     "'"V'inerer that
     I hade shooking in a sump

lstm_trial:1, iter:8399/20000 loss:1.4742884504795075
generated sequence: Where-'that which do nothing in the line to seer,
     but go, I have to his pracolen, that that as y

lstm_trial:1, iter:8499/20000 loss:1.5264128279685973
generated sequence: What I thought came. I so but, that elome that oppon lorgess that not ruture rejurcemence bettae comp

lstm_trial:1, iter:8599/20000 loss:1.5337183344364167
generated sequence: Wat Yancent
     bititrers in the stall bean liftencied, but round glivere was possed for a ney upon 

lstm_trial:1, iter:8699/20000 loss:1.5097940158843994
generated sequence: We
     which Yasce take, not. First,
     can reswous my my Pusone. yIt at plome
     under day. The

lstm_trial:1, iter:8799/20000 loss:1.4879199743270874
generated sequence: Whisk go. It whit' unon accett ammeating
 

lstm_trial:1, iter:12999/20000 loss:1.4915334141254426
generated sequence: What is a hand-feet at hordevus rably plank when he
     of
     after of
     an hope aw of it
     

lstm_trial:1, iter:13099/20000 loss:1.4621887493133545
generated sequence: Waving of the
     duce, that this sutch as a plecked, it. she would loage his siliard.

     McMurde

lstm_trial:1, iter:13199/20000 loss:1.4796026933193207
generated sequence: What a little leage of the puzzling the open."

     "She awas blit supporially caming. If you are
  

lstm_trial:1, iter:13299/20000 loss:1.423561372756958
generated sequence: Which an unnir, plon. There was tray-bounting Mis Merisy something all that it is see the hunrumed wa

lstm_trial:1, iter:13399/20000 loss:1.4731184005737306
generated sequence: Wat. That were he unandent it untid upon the closed a
     dlink cle alservo," and
     as to we
    

lstm_trial:1, iter:13499/20000 loss:1.4951421880722047
generated sequence: We used, that in posting me bexals, 

lstm_trial:1, iter:17699/20000 loss:1.417267206311226
generated sequence: We had shouldals and came in side entered are no order"--Ged
     the Abrighted daked the
     minual

lstm_trial:1, iter:17799/20000 loss:1.4342951357364655
generated sequence: Westing and, there' I fear was so begch into the
     work of the
     had really get involage, I of 

lstm_trial:1, iter:17899/20000 loss:1.4498785281181334
generated sequence: Wery of precervamed
     upon the habord, those Lack, and how. To may, which spivioused, Ind." Hear."

lstm_trial:1, iter:17999/20000 loss:1.4390620815753936
generated sequence: We spoken to be to sucher of Tul'gs you met the currorsular In much me alausty attentine inscribue ov

lstm_trial:1, iter:18099/20000 loss:1.4314140915870666
generated sequence: Which within ways guiding opened to an expect, sude, with hide?"

     "No, and she begon myself you


lstm_trial:1, iter:18199/20000 loss:1.466660544872284
generated sequence: Wathous object of could not have righ

KeyboardInterrupt: 

In [ ]:
plt.title("Average Loss Over 20,000 Iterations")
plt.xlabel('iters')
plt.ylabel('loss')
plt.plot(mean_losses_rnn)
plt.plot(mean_losses_lstm)
plt.legend(['RNN', 'LSTM'])
plt.show()

In [ ]:
print(eval_step(net_rnn, predicted_len=600))

In [ ]:
print(eval_step(net_lstm, predicted_len=600))

In [ ]:
print(cv_results.reset_index().to_markdown(index=False))

In [ ]:
fig.write_html('fig2.html', include_plotlyjs='cdn')